In [15]:
def g_0(n):
    return ("?",) * n
def s_0(n):
    return ("T",) * n 
# NOTE: feel free to use a different symbol
# in case of encoding issues

In [16]:
def more_general(h1, h2):
    """
    >>> more_general(("sunny", "warm", "?"), ("sunny", "?", "normal"))
    False
    
    """
    # Check if a is more general than b
    # else return False
    result = [True if a == "?" or a == b or b == "T" else False for a,b in zip(h1, h2)]
    for a in result:
        if a is not True:
            return a
    return True

In [17]:
def min_generalizations(h, x):
    """
    >>> min_generalizations(h=("sunny", "warm"), x=("rainy", "warm"))
    [('?', 'warm')]
    
    """
    # Switch attribute value to Wild card if contradict
    result = ["?" if a != "T" and a != b  else b for a,b in zip(h, x)]
    return [tuple(result)]

In [20]:
def min_specializations(h, domains, x):
    """
    >>> g = ('sunny', '?', '?', '?', '?', '?', '?')
    >>> d = [['sunny', 'rainy', 'cloudy'],['warm', 'cold'],['normal', 'high'],['strong', 'weak'],['warm', 'cool'],['same', 'change'],['long', 'medium', 'short']]
    >>> x = ('sunny',  'warm', 'high', 'strong', 'cool', 'change', 'long')
    >>> min_specializations(g, d, x)
    [('T', '?', '?', '?', '?', '?', '?'), ('sunny', 'cold', '?', '?', '?', '?', '?'), ('sunny', '?', 'normal', '?', '?', '?', '?'), ('sunny', '?', '?', 'weak', '?', '?', '?'), ('sunny', '?', '?', '?', 'warm', '?', '?'), ('sunny', '?', '?', '?', '?', 'same', '?'), ('sunny', '?', '?', '?', '?', '?', 'medium'), ('sunny', '?', '?', '?', '?', '?', 'short')]
     
    >>> min_specializations(h=("?", "x"), domains=[["a", "b", "c"], ["x", "y"]], x=("b", "x"))
    [('a', 'x'), ('c', 'x'), ('?', 'T')]
    
    """
    
    # create more specialise hypothesis using domian
    result = []
    h = list(h)
    for i in range(len(h)):
        if h[i] == "?":
            for a in domains[i]:
                if a != x[i]:
                    h_new = h.copy()
                    h_new[i] = a
                    result.append(tuple(h_new))
        elif h[i] == x[i]:
            h_new = h.copy()
            h_new[i] = "T"
            result.append(tuple(h_new))
    return result

In [21]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=7)

In [6]:
from io import StringIO
import pandas as pd

# create table of examples
examples = pd.read_csv( StringIO("""Sky,Temp,Humid,Wind,Water,Forecast,Sleep,EnjoySport
sunny,warm,normal,strong,warm,same,long,True
sunny,warm,high,strong,warm,same,long,True
rainy,cold,high,weak,warm,change,medium,False 
sunny,warm,high,strong,cool,change,long,True
cloudy,cold,high,strong,cool,change,short,False"""),converters={'EnjoySport': lambda x: x=='True'})

print(examples)

      Sky  Temp   Humid    Wind Water Forecast   Sleep  EnjoySport
0   sunny  warm  normal  strong  warm     same    long        True
1   sunny  warm    high  strong  warm     same    long        True
2   rainy  cold    high    weak  warm   change  medium       False
3   sunny  warm    high  strong  cool   change    long        True
4  cloudy  cold    high  strong  cool   change   short       False


In [22]:
def consistent(h,x):
    """
    >>>consistent(("sunny", "warm", "?"), ("sunny", "warm", "normal"))
    True
    >>>consistent(("sunny", "warm", "T"), ("sunny", "warm", "normal"))
    False
    >>>consistent(("sunny", "warm", "high"), ("sunny", "warm", "normal"))
    False
    
    """
    result = [True if a == "?" or (a == b and b != "T") else False for a,b in zip(h,x)]
    for a in result:
        if a is not True:
            return a
    return True

In [24]:
def setDomains(examples):
    """
    >>>setDomains(examples.T[:-1].T.values)
    [['sunny', 'rainy', 'cloudy'], ['warm', 'cold'], ['normal', 'high'], ['strong', 'weak'], ['warm', 'cool'], ['same', 'change'], ['long', 'medium', 'short']]
 
    """
    # Filter the unique attribute value
    domains = [[i] for i in examples[0]]
    for x in examples:
        for i in range(len(x)):
            add = None
            for j in domains[i]:
                if j == x[i]:
                    add = None
                    break
                else:
                    add = x[i]
            if add is not None:
                domains[i].append(x[i])            
    return domains

In [31]:
def candidate_elimination(examples):
    # Filter out domain from examples
    domains = setDomains(examples.T[:-1].T)
    
    # Define most general and specific case 
    n = len(domains)
    G = [g_0(n)]
    S = [s_0(n)]
    
    for x in examples:
        
        #  x[-1] extract the target value
        if x[-1]:
            x = x[:-1]
            
            # Remove out any hypothesis is not consistent 
            for g in G:
                if not consistent(g,x):
                    G.remove(g)
            for s in S:
                if not consistent(s,x):
                    S.remove(s)
                S_new = min_generalizations(s,x)
                for s_new in S_new:
                    result = True
                    for g in G:
                        if not more_general(g, s_new):
                            result = False
                            break
                    if consistent(s_new,x) and result:
                        S.append(s_new)
                S_remove = []
            for s in S:
                S_diff = S.copy()
                S_diff.remove(s)
                    
                for s_diff in S_diff:
                    if more_general(s, s_diff):
                        S_remove.append(s)
                        break
            for s_remove in S_remove:
                S.remove(s_remove)
                    
        else:
            #  x[-1] extract the target value
            x = x[:-1]
            
            # Remove out any hypothesis is not consistent 
            for s in S:
                if consistent(s,x):
                    S.remove(s)
                    
            # Remove out any hypothesis is not consistent
            # minimumum specialise the previous hypothesis 
            for g in G:
                if consistent(g,x):
                    G.remove(g)
                    G_new = min_specializations(g, domains, x)
                    for g_new in G_new:
                        result = True
                        
                        # Validate new hypothesis
                        if not consistent(g_new,x):
                            for s in S:
                                if not more_general(g_new,s):
                                    result = False
                                    break
                        if result:
                            G.append(g_new)
                G_remove = []
            for g in G:
                G_diff = G.copy()
                G_diff.remove(g)
                    
                for g_diff in G_diff:
                    if more_general(g, g_diff):
                        G_remove.append(g)
                        break
            for g_remove in G_remove:
                G.remove(g_remove) 
    
        print("G: ", G)
        print("S: ", S)
        print()
                    
    return G, S

In [32]:
G,S = candidate_elimination(examples.values)

G:  [('?', '?', '?', '?', '?', '?', '?')]
S:  [('sunny', 'warm', 'normal', 'strong', 'warm', 'same', 'long')]

G:  [('?', '?', '?', '?', '?', '?', '?')]
S:  [('sunny', 'warm', '?', 'strong', 'warm', 'same', 'long')]

G:  [('sunny', '?', '?', '?', '?', '?', '?'), ('?', 'warm', '?', '?', '?', '?', '?'), ('?', '?', '?', 'strong', '?', '?', '?'), ('?', '?', '?', '?', '?', 'same', '?'), ('?', '?', '?', '?', '?', '?', 'long')]
S:  [('sunny', 'warm', '?', 'strong', 'warm', 'same', 'long')]

G:  [('sunny', '?', '?', '?', '?', '?', '?'), ('?', 'warm', '?', '?', '?', '?', '?'), ('?', '?', '?', 'strong', '?', '?', '?'), ('?', '?', '?', '?', '?', '?', 'long')]
S:  [('sunny', 'warm', '?', 'strong', '?', '?', 'long')]

G:  [('sunny', '?', '?', 'strong', '?', '?', '?'), ('?', 'warm', '?', 'strong', '?', '?', '?'), ('?', '?', '?', 'strong', '?', '?', 'long')]
S:  [('sunny', 'warm', '?', 'strong', '?', '?', 'long')]



In [33]:
def relax(h1, h2):
    # Try relax from S space to identify more hypotheis between G and S space
    h1 = list(h1)
    h2 = list(h2)
    result = []
    for i in range(len(h1)):
        if h1[i] == "?"  and h2[i] != "?":
            h_new = h1.copy()
            h_new[i] = h2[i]
            result.append(tuple(h_new))
    return result

In [34]:
def match(h1, h2):
    result = [True if a == b else False for a,b in zip(h1,h2)]
    for a in result:
        if a is not True:
            return a
    return True

In [35]:
def ver_space(G,S):
    
    # Collect all possible hypothesis between G and S pace
    VS = []
    for g in G:
        for s in S:
            Result = relax(g, s)
            for result in Result:
                add = False
                if len(VS) > 0:
                    for vs in VS:
                        if match(result, vs):
                            add = False
                            break
                        else:
                            add = True
                else:
                    VS.append(result)
                if add:
                    VS.append(result)
    VS.extend(G)
    VS.extend(S)
    return VS

In [36]:
VS = ver_space(G,S)

In [37]:
def check(examples, VS):
    check=[]
    
    # Validate if version space can match with Ground Truth
    for x in examples:
        if x[-1]:
            result = sum([consistent(h,x[:-1]) for h in VS])
            check.append(result)
        else:
            result = sum([not consistent(h,x[:-1]) for h in VS])
            check.append(result)
    return check

In [38]:
examples['score h in VS'] = check(examples.values, VS)
print(examples)

      Sky  Temp   Humid    Wind Water Forecast   Sleep  EnjoySport  \
0   sunny  warm  normal  strong  warm     same    long        True   
1   sunny  warm    high  strong  warm     same    long        True   
2   rainy  cold    high    weak  warm   change  medium       False   
3   sunny  warm    high  strong  cool   change    long        True   
4  cloudy  cold    high  strong  cool   change   short       False   

   score h in VS  
0              7  
1              7  
2              7  
3              7  
4              7  
